In [21]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [22]:
#načítanie datasetu
data = pd.read_csv("dataset_stroke.csv")

In [23]:
data

,id,gender,age,hypertension,heart_disease,ever_married,work_type,Residence_type,avg_glucose_level,bmi,smoking_status,stroke
0,9046,Male,67.0,0,1,Yes,Private,Urban,228.69,36.6,formerly smoked,1
1,51676,Female,61.0,0,0,Yes,Self-employed,Rural,202.21,NaN,never smoked,1
2,31112,Male,80.0,0,1,Yes,Private,Rural,105.92,32.5,never smoked,1
3,60182,Female,49.0,0,0,Yes,Private,Urban,171.23,34.4,smokes,1
4,1665,Female,79.0,1,0,Yes,Self-employed,Rural,174.12,24.0,never smoked,1
...,...,...,...,...,...,...,...,...,...,...,...,...
5105,18234,Female,80.0,1,0,Yes,Private,Urban,83.75,NaN,never smoked,0
5106,44873,Female,81.0,0,0,Yes,Self-employed,Urban,125.20,40.0,never smoked,0
5107,19723,Female,35.0,0,0,Yes,Self-employed,Rural,82.99,30.6,never smoked,0
5108,37544,Male,51.0,0,0,Yes,Private,Rural,166.29,25.6,formerly smoked,0


In [24]:
#z datasetu je zrejmé, že posledná osoba ktorá mala mrtvicu má "id" 43424
#ked si danú osobu vypíšeme, zistíme, že prvých 249 záznamov v datasete malo mrtvicu (potom sú už len 0)
last_person_with_stroke = data[data["id"]==43424]
last_person_with_stroke

,id,gender,age,hypertension,heart_disease,ever_married,work_type,Residence_type,avg_glucose_level,bmi,smoking_status,stroke
248,43424,Female,78.0,0,0,Yes,Private,Rural,78.81,19.6,Unknown,1


In [25]:
#zahodíme nepotrebné stĺpce, aby sme pracovali len s faktormi, ktoré nás zaujímajú
data = data.drop(data.columns[[0, 6, 7, 8, 9, 10]], axis=1)

In [26]:
data

,gender,age,hypertension,heart_disease,ever_married,stroke
0,Male,67.0,0,1,Yes,1
1,Female,61.0,0,0,Yes,1
2,Male,80.0,0,1,Yes,1
3,Female,49.0,0,0,Yes,1
4,Female,79.0,1,0,Yes,1
...,...,...,...,...,...,...
5105,Female,80.0,1,0,Yes,0
5106,Female,81.0,0,0,Yes,0
5107,Female,35.0,0,0,Yes,0
5108,Male,51.0,0,0,Yes,0


In [27]:
#zoberieme prvých 496 záznamov, polovica mala mrtvicu, druhá nie
data = data.head(496)

In [28]:
data

,gender,age,hypertension,heart_disease,ever_married,stroke
0,Male,67.0,0,1,Yes,1
1,Female,61.0,0,0,Yes,1
2,Male,80.0,0,1,Yes,1
3,Female,49.0,0,0,Yes,1
4,Female,79.0,1,0,Yes,1
...,...,...,...,...,...,...
491,Male,60.0,0,0,Yes,0
492,Male,74.0,0,0,Yes,0
493,Female,23.0,0,0,No,0
494,Female,55.0,0,0,Yes,0


In [29]:
#zistíme najmenší vek respondenta = 0.64 rokov
data[data["age"]==data["age"].min()]

,gender,age,hypertension,heart_disease,ever_married,stroke
363,Female,0.64,0,0,No,0


In [30]:
#zistíme najväčší vek respondenta = 82 rokov
data[data["age"]==data["age"].max()]

,gender,age,hypertension,heart_disease,ever_married,stroke
23,Male,82.0,0,1,Yes,1
32,Female,82.0,1,0,Yes,1
35,Female,82.0,1,1,No,1
42,Male,82.0,0,1,Yes,1
56,Female,82.0,0,0,Yes,1
128,Male,82.0,0,0,Yes,1
140,Male,82.0,0,1,Yes,1
187,Female,82.0,1,1,Yes,1
188,Male,82.0,0,0,Yes,1
337,Female,82.0,0,1,No,0


In [70]:
#preštylizujeme si dáta, kedže neuronová sieť nevie pracovať so Stringami
#pohlavie bude 1 ak je to muž, v opačnom prípade bude 0
#to či bol človek ženatý bude 1 ak áno, ak nie tak 0
data.loc[data['gender'] == "Male", 'gender'] = 1
data.loc[data['gender'] == "Female", 'gender'] = 0
data.loc[data['ever_married'] == "Yes", 'ever_married'] = 1
data.loc[data['ever_married'] == "No", 'ever_married'] = 0
data

C:\Users\Andrej\anaconda3\lib\site-packages\pandas\core\indexing.py:1765: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(loc, value)


,gender,age,hypertension,heart_disease,ever_married,stroke
0,1,67.0,0,1,1,1
1,0,61.0,0,0,1,1
2,1,80.0,0,1,1,1
3,0,49.0,0,0,1,1
4,0,79.0,1,0,1,1
...,...,...,...,...,...,...
491,1,60.0,0,0,1,0
492,1,74.0,0,0,1,0
493,0,23.0,0,0,0,0
494,0,55.0,0,0,1,0


In [32]:
#vyberieme si X-sové hodnoty (vstupy)
X = data.iloc[:, [0,1,2,3,4]].values

In [33]:
X

array([[1, 67.0, 0, 1, 1],
       [0, 61.0, 0, 0, 1],
       [1, 80.0, 0, 1, 1],
       ...,
       [0, 23.0, 0, 0, 0],
       [0, 55.0, 0, 0, 1],
       [0, 71.0, 0, 0, 1]], dtype=object)

In [34]:
#vyberieme požadovaný výstup
Y = data.iloc[:, 5].values

In [35]:
Y

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

In [48]:
#vytvoríme si testovacie a trénovacie dáta, pomocou knižnice sklearn
from sklearn.model_selection import train_test_split
X_Train, X_Test, Y_Train, Y_Test = train_test_split(X, Y, train_size=0.8, random_state=1)

In [49]:
from sklearn.preprocessing import StandardScaler

In [50]:
from sklearn.svm import SVC

In [51]:
#vytvoríme list s presnostami
list = []

In [52]:
sc_X = StandardScaler()
X_Train = sc_X.fit_transform(X_Train)
X_Test = sc_X.fit_transform(X_Test)

#vytvoríme si klasifikátor, ktorý nám dáta rozdelí
#aktivačná funkcia je nastavená na linearnu
#random_state je nastavený na 1, čiže sa údaje pomiešajú náhodne
classifier = SVC(kernel="linear", random_state=1, tol=0.01)
#natrénujeme si neuronovú sieť pomocou príkazu fit
classifier.fit(X_Train, Y_Train)

score = classifier.score(X_Test,Y_Test)
list.append(score)

In [53]:
list

[0.67]

In [54]:
Y_Pred = classifier.predict(X_Test)
finalScore = classifier.score(X_Test, Y_Test)

In [56]:
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(Y_Test, Y_Pred)

In [57]:
from matplotlib.colors import ListedColormap
X_Set, Y_Set = X_Train, Y_Train
#doplnit vizualizaciu

In [58]:
X_Set, Y_Set = X_Test, Y_Test
#doplnit vizualizaciu

In [65]:
def vratPredikciu(pohlavie, vek, hypertenzia, choroby_srdca, zenaty, finalScore):
    dataset = pd.DataFrame({"gender":pohlavie,"age":vek,"hypertension":hypertenzia,"heart_disease":choroby_srdca,"ever_married":zenaty},index=[0])
    X = sc_X.transform(dataset)
    classificationPrediction = classifier.predict(X)
    skore = finalScore*100
    if(classificationPrediction[0]==0):
        return "Nedostane, Presnost: "+str(skore)+"%"
    else:
        return "Dostane, Presnost: "+str(skore)+"%"

In [69]:
vratPredikciu(1,25,1,0,0,finalScore)

'Nedostane, Presnost: 67.0%'